In [1]:
import mlflow
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
import mlflow_logging

In [3]:
class DummyObject:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)

FLAGS=DummyObject(input_file='',
                 max_words=1000,
                 batch_size=16,
                 epochs=5,
                 num_classes=2,
                 learning_rate=None,
                 max_seq_length=120)

In [4]:
logging = mlflow_logging.TensorflowAutoLogging('custom_loging_exp','test1',1)

In [5]:
logging.set_git_params('.')

In [6]:
logging.set_notes('Just for testing')

In [7]:
logging.set_other_params(FLAGS.__dict__)

In [ ]:
max_words = 1000
batch_size = 16
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Dense(2020, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Loading data...
25000 train sequences
25000 test sequences
2 classes
Vectorizing sequence data...
x_train shape: (25000, 1000)
x_test shape: (25000, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (25000, 2)
y_test shape: (25000, 2)
Building model...


/Users/ankur.kumar/tf2_env/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
/Users/ankur.kumar/tf2_env/lib/python3.7/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 10s 453us/sample - loss: 0.4000 - accuracy: 0.8291 - val_loss: 0.3375 - val_accuracy: 0.8604
Epoch 2/5
21520/22500 [===========================>..] - ETA: 0s - loss: 0.3324 - accuracy: 0.8574

In [ ]:
logging.stop_logging()